In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# 采用knn检测函数进行特征匹配
# RANSAC算法来计算透视变换矩阵 
def keyPoints(keypoints_1, keypoints_2, features_1, features_2, ratio, reprojThresh):
    # 创建BFMatcher对象
    matcher = cv2.BFMatcher()
    # 使用BFMatcher进行特征点匹配
    rawMatches = matcher.knnMatch(features_1, features_2, 2)

    # 受cv2.findHomography()函数输入限制，更改类型
    keypoints_1 = np.float32([kp.pt for kp in keypoints_1])
    keypoints_2 = np.float32([kp.pt for kp in keypoints_2])

    matches = []
    for m in rawMatches:
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            matches.append((m[0].trainIdx, m[0].queryIdx))

    # 如果筛选后的匹配特征点对数量大于 4，则将这些特征点对转换为 NumPy 数组
    if len(matches) > 4:
        ptsA = np.float32([keypoints_1[i] for (_, i) in matches])
        ptsB = np.float32([keypoints_2[i] for (i, _) in matches])

        # 用 RANSAC 算法来计算透视变换矩阵 H，并返回变换矩阵、匹配特征点对和状态信息。
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

        return (matches, H, status)

    else:
        print("可匹配的特征点数量较低")

def features(gray_image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)

    return keypoints, descriptors


# 特征匹配
def Match(feature_1, feature_2):
    bf = cv2.BFMatcher()
    rawMatches = bf.knnMatch(feature_1, feature_2, k=2)

    matches = []
    # 过滤
    for m, n in rawMatches:
        if m.distance < 0.75 * n.distance:
            matches.append(m)

    return matches


#  读入图片
def img(image_path):
    image = cv2.imread(image_path)
    img = cv2.resize(image, (1028, 762))
    return img



def show_img(name, img):
    # 获取图片时转换为RGB格式，显示图片则需要转换为BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    plt.imshow(img)
    plt.text(img.shape[1] // 2, img.shape[0] + 150, name, color='red', fontsize=12, ha='center')
    cv2.imwrite('./work_test/res_' + name + '.jpg', img)
    plt.show()


# 拼接显示
def show(image1, image2):
    image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    # 水平拼接两张图像
    concatenated_image = np.hstack((image1, image2))

    # 显示拼接后的图像
    plt.imshow(concatenated_image)
    plt.axis('off')

    cv2.imwrite('./work_test/result_2.jpg', concatenated_image)
    


if __name__ == "__main__":

    image_path_1 = './left2.jpg'
    image_path_2 = './right2.jpg'
    # 读取图片
    image_1 = img(image_path_1)
    image_2 = img(image_path_2)
    # 特征点，描述符
    k_1, d_1 = features(image_1)
    k_2, d_2 = features(image_2)

    # 显示图片的关键点
    match = Match(d_1, d_2)
    # 绘制K近邻匹配结果
    image_with_match = cv2.drawMatches(image_1, k_1, image_2, k_2, match, None,
                                       flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    show_img('Image with matches', image_with_match)

    M = keyPoints(k_2, k_1, d_2, d_1, ratio=0.75, reprojThresh=4.0)
    (matches, H, status) = M
    print('两张图片的单应性矩阵：\n', H)
    result = cv2.warpPerspective(image_2, H,(image_1.shape[1] + image_2.shape[1], max(image_1.shape[0], image_2.shape[0])))
    show_img('Result', result)